**Initializing libiraries:**

In [1]:
!pip install openai
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

**Ask Chatbot**

In [4]:
def load_chatbot():
    model_name = "microsoft/DialoGPT-medium"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return tokenizer, model

def chat():
    tokenizer, model = load_chatbot()
    chat_history_ids = None

    while True:
        user_input = input("You: ")
        if not user_input:
            continue

        new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt").to(model.device)
        chat_history_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

        response_ids = model.generate(
            chat_history_ids,
            max_length=chat_history_ids.shape[-1] + 50,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.6,
            top_k=40,
            top_p=0.85,
            repetition_penalty=1.2,
        )

        print("Chatbot:", tokenizer.decode(response_ids[:, chat_history_ids.shape[-1]:][0], skip_special_tokens=True), "\n")

chat()

You: Hello
Chatbot: Hi! :D 

You: You are a chatbot, remember that
Chatbot: I'm not a bot, I just like to talk. 

You: Ok
Chatbot: I'm not sure if you're being serious or not, but I'll just say this : You have been banned from r pyongyang. 

You: what's that
Chatbot: A bot that responds to you in the form of a message. 



KeyboardInterrupt: Interrupted by user